In [1]:
import geopandas as gpd
import pandas as pd
import pandas
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
from pyproj import Proj, Transformer

In [2]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [3]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['intersected'] = None
road['mother_road']='None'

buildings['road_direction'] = 'None'
buildings['distance2_intersecting_road'] = 0
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [4]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def addDirection_BuildingCode(direction,length,index,road_index):
    #right-even, left-odd number 
    length=int(length)
    # print(length)
    if length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}
        motherroad_start_point = {'col1': ['name1'], 'geometry': [Point(mother_road.start_point_x, mother_road.start_point_y)]}
        
        gdf = gpd.GeoDataFrame(d, crs=4326)
        mother_df = gpd.GeoDataFrame(motherroad_start_point, crs=4326)
        
        sp_point=gdf.iloc[0]
        mother_st_p=mother_df.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                mother_line=l
                break

        result=split(mother_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            splitted_line.srid = 4326
            if splitted_line.contains(mother_st_p.geometry):
                break
            else:
                continue
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
        
        buildings.at[index,'road_length_d_code']=str(int(geom_transformed.length))+"/"
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(length)))
        else:
            # print("On  Line")

            buildings.at[index,'road_direction']="on_line"
    else :
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(length)))
        else:
            # print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    print(buildings.iloc[index].road_length_d_code)
    
#     print(buildings.at[index,'road_length_d_code'])
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(bearing,index):
    # default direction is west to east , south to north 
    # print("ma revise garna aako dubai vettera")
    if bearing is None :
        bearing=0
    bearing=int(bearing)
    def assign_start():
        print("start")
        
        road.at[index,'mother_road']='start'

    def assign_end():
    # east_west/west_east direction , assign to east
        print("end")
        road.at[index,'mother_road']='end'

    
    if ((bearing >= 0) and (bearing<=45)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
        
    if ((bearing > 45) and (bearing<135)) :
        
        # east_west/west_east direction , assign to east
        assign_end()
    
    if ((bearing >= 135) and (bearing<=225)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
    if ((bearing > 225) and (bearing<315)) :
            
        # east_west/west_east direction , assign to east
        assign_end()
    if ((bearing >= 315) and (bearing<=360)) :
    #    North_South/south_north Direction, assign to south
        
        assign_start()      

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            if line.distance(start_point) < 1e-8 :
                # print("start found "+ str(row.NEW))
                # print("mero start vetiyo guys ")
                road.at[index_road,'start']=str(row.NEW)
    
            if line.distance(end_point) < 1e-8:
                # print("end found "+ str(row.NEW))
                road.at[index_road,'end']=str(row.NEW)
                # print("mero end vetiyo guys ")


def scan_I_feature(line_row,index_road):
    
        if ((line_row.start!= None) and (line_row.end== None)) :
            road.at[index_road,'mother_road']='start'
        if ((line_row.start== None) and (line_row.end != None)) :
            road.at[index_road,'mother_road']='end'
    
        if ((line_row.start!= None) and (line_row.end != None)) :
        #both start and end exist which means line is in between roads 
            road.at[index_road,'mother_road']='None'
            ReviseStartEndDateWithBearing(line_row.bearing,index_road)

                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False    
    
def AssignIntersectedPoint(start_point_assumed,end_point_assumed,row,index):
    if  row.mother_road :
        mother_road_direction = row.mother_road
        mother_road_code=None
        if mother_road_direction == 'start':
            mother_road_code=row.start
        elif mother_road_direction == 'end' :
            mother_road_code=row.end
        mother_road= road.loc[road['NEW'] == mother_road_code].geometry
        current_road = row.geometry
        mother_road_geom=mother_road
        for l in mother_road:
            for single in l:
                mother_road_geom=single
            break
#         print(mother_road_geom)
        for l in current_road:
            road_geom=l
            break
#         print(road_geom)
        start_point=mother_road_geom.intersection(road_geom)
        st=str(type(start_point))
#         print(st)
#         geoseries=f"""<class 'geopandas.geoseries.GeoSeries'>"""
#         print(geoseries)
    
        if isinstance(start_point,pandas.core.series.Series) :
            print(type(start_point))
            print('not counted')
            
            road.at[index,'intersected']='No'
        else:
            print(start_point)
            try:
                x=start_point.x
                y=start_point.y
                road.at[index,'start_point_x']=float(x)
                road.at[index,'start_point_y']=float(y)              
            except:
                try:
                    for p in start_point:
                        x=p.x
                        y=p.y
                        break
                    road.at[index,'start_point_x']=x
                    road.at[index,'start_point_y']=y
                except:
                    try:
                        df_line = start_point.boundary
                        start=df_line[0]
                        road.at[index,'start_point_x']=start.x
                        road.at[index,'start_point_y']=start.y
                    except:
                        pass
#                     road.at[index,'start_point_x']=0
#                     road.at[index,'start_point_y']=0
 

        
def get_building_projected_road_length(road_linked_wrt_point,row):
    road_code = row.NEW
#     print(road_linked_wrt_point)
    linked_road_geometry = road_linked_wrt_point.geometry
    building_geometry = row.geometry
    degree_length_for_building=linked_road_geometry.project(building_geometry)
    # 5m=0.00005 in degree 
    building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always 
    for r in building_point_projected_on_road:
        building_point_projected_on_road_single_geom=r
        break
    try:
        projected_building_on_road = {'col1': ['name1'], 'geometry': [Point(building_point_projected_on_road_single_geom.x, building_point_projected_on_road_single_geom.y)]}
    except:
        print(projected_building_on_road)
        raise("found error")

    road_link_wrt_point_x=road_linked_wrt_point.start_point_x
    road_link_wrt_point_y=road_linked_wrt_point.start_point_y
    if road_link_wrt_point_x!=0:
        try:
            road_start_point = {'col1': ['name1'], 'geometry': [Point(road_link_wrt_point_x,road_link_wrt_point_y)]}

            projected_building_on_road_gdf = gpd.GeoDataFrame(projected_building_on_road, crs=4326)


            road_start_df = gpd.GeoDataFrame(road_start_point, crs=4326)


            splitting_point=projected_building_on_road_gdf.iloc[0]
            start_point=road_start_df.iloc[0]

            for line in linked_road_geometry:
                # print(line)
                for l in line:
                    mother_line=l
                    break
            result=split(mother_line,splitting_point.geometry.buffer(0.0000001))
    #         print(result)
    #         print(start_point.geometry)
            for r  in result:
                    splitted_line=r
                    splitted_line.srid = 4326
                    if splitted_line.contains(start_point.geometry.buffer(0.0000001)) != True:
    #                     print("i contain")
                        break
                    else:
                        continue


            my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
            geom_transformed = ops.transform(my_transformer.transform, splitted_line)

            clipped_length=geom_transformed.length
            return clipped_length ,splitted_line ,building_point_projected_on_road_single_geom,start_point.geometry
        except:
            pass
 
        
def CalculateTouchLine():
    for index,row in road.iterrows():
        # print(row)
        start_point,end_point=getStartEndPoint(row,"road")
        addStartEndPoint(start_point,end_point,index)
        CalculateBearing(start_point,end_point,index)
        FindIntersectedFeatures(start_point,end_point,row,index)
        scan_I_feature(row,index)
        AssignIntersectedPoint(start_point,end_point,row,index)
        print(index)
        # break
    print("Sucessfully added : startpoint, endpoint , Start touching Line , End Touching Line , bearing ")

def assign_building_code():  
    for index, row in buildings.iterrows():
        print(index)
        if(index>0):   
            print(buildings.iloc[index-1].road_length_d_code)
            
        road_linked_wrt_point= road.loc[road['NEW'] == row.NEW]
        try:
            clipped_length,splitted_mother_road_side,building_point_projected_on_road_single_geom,start_point=get_building_projected_road_length(road_linked_wrt_point,row)

            splitted_with_buffer_boi=split(splitted_mother_road_side,building_point_projected_on_road_single_geom.buffer(0.0001)) 

            for r  in splitted_with_buffer_boi:
                roi_line=r
                roi_line.srid = 4326
                if roi_line.contains(start_point.buffer(0.0000001)):

                    break
                else:
                    continue

            df_line = roi_line.boundary
            start_roi,end_roi=df_line[0],df_line[1]
            if (round(start_roi.x,6) == round(building_point_projected_on_road_single_geom.x,6)) and (round(start_roi.y,6) == round(building_point_projected_on_road_single_geom.y,6)) :
                start_roi=end_roi
                end_roi=building_point_projected_on_road_single_geom

            try:
                road_index=road_linked_wrt_point.index[0]
                road.at[road_index,'length']=length[0]       
                start = bdcPoint()
                end = bdcPoint()
                point = bdcPoint()
                poi = row.geometry
                try:
                    start.x,start.y,end.x,end.y,point.x,point.y=start_roi.x,start_roi.y,end_roi.x,end_roi.y,poi.x,poi.y

                    direction=directionOfPoint(start,end,point)
                    # print(direction)
                    # break
                    addDirection_BuildingCode(direction,clipped_length,index,road_index)
                    print("added building code")
                except:
                    # break
                    continue
            except:
                continue
        except:
            continue
        break
         
        
    print("-------Sucess ------ added: road_length_d_code , length of road , direction of building ")

    
                   


In [15]:
CalculateTouchLine()

<class 'geopandas.geoseries.GeoSeries'>
not counted
0
end
POINT (83.47182876258739 27.59323857200252)
1
POINT (83.47268607010614 27.60246986818993)
2
POINT (83.47040628242405 27.57787412609008)
3
POINT (83.47172783257349 27.59225408505802)
4
end
POINT (83.46736454413843 27.56698659255727)
5
POINT (83.47153400529015 27.59036561757927)
6
end
POINT (83.47187124206425 27.59364565192533)
7
end
POINT (83.47263962641767 27.60196558594311)
8
POINT (83.47213499163661 27.59644264511201)
9
POINT (83.47071132616685 27.58135272534804)
10
end
POINT (83.47292715316564 27.60530247452965)
11
POINT (83.47216569628989 27.59679215853134)
12
end
POINT (83.47224355599536 27.59769741250562)
13
POINT (83.47224012238382 27.59758753693535)
14
POINT (83.47163945439735 27.5913920255216)
15
end
POINT (83.47287071800929 27.60470846242566)
16
end
POINT (83.47069512038354 27.58112225878529)
17
POINT (83.46663855382627 27.56464723648082)
18
POINT (83.47222943933724 27.59751778281947)
19
POINT (83.4691354810177 27.5728

end
POINT (83.47074640152528 27.59777585766977)
162
POINT (83.45999136960609 27.58456990877431)
163
end
POINT (83.45342890062335 27.58687694231708)
164
POINT (83.45514321348855 27.59225771202387)
165
POINT (83.46122202617562 27.5841100791169)
166
start
POINT (83.4581126966437 27.57306709237247)
167
start
POINT (83.45887908360288 27.57336984104541)
168
start
POINT (83.46079436976441 27.58428014900971)
169
POINT (83.43969330140482 27.59592745678674)
170
POINT (83.45864736968042 27.5734361417646)
171
POINT (83.43984491810852 27.59583754436721)
172
POINT (83.45459247316415 27.59034999995259)
173
end
POINT (83.46031340064297 27.5741794943882)
174
POINT (83.45914828396621 27.57330037741053)
175
POINT (83.45997992213574 27.58453232250872)
176
start
POINT (83.45543963812884 27.5938612392049)
177
start
POINT (83.43780754538199 27.59591046139872)
178
end
POINT (83.45892516306589 27.56683897154136)
179
start
POINT (83.43251684727471 27.60744073140467)
180
POINT (83.43736914746825 27.5958884028276

end
POINT (83.46663768687978 27.59885725016204)
323
POINT (83.46625438143462 27.60141547154683)
324
POINT (83.46631128783571 27.60160856858062)
325
POINT (83.46243477274663 27.59057015668742)
326
POINT (83.46287478944004 27.59382176706094)
327
POINT (83.46807054521952 27.59966902230758)
328
POINT (83.46774817423892 27.59682975289081)
329
<class 'geopandas.geoseries.GeoSeries'>
not counted
330
start
MULTIPOINT (83.46445790830586 27.60520732895515, 83.46449663311319 27.60546659270915)
331
end
POINT (83.4690590062682 27.60068633810408)
332
POINT (83.46164686680953 27.5764183161524)
333
POINT (83.46701540303849 27.60569154822645)
334
end
POINT (83.46932860682978 27.5950278298705)
335
POINT (83.46263721373646 27.59217533682238)
336
MULTIPOINT (83.46307715488683 27.58208449560306, 83.46318779937752 27.58234113603464)
337
POINT (83.46671072981638 27.59917494916465)
338
POINT (83.466986702974 27.58818580303245)
339
POINT (83.46182581840685 27.57691377335203)
340
end
MULTIPOINT (83.465773147013

POINT (83.43381872205646 27.58632225576753)
479
<class 'geopandas.geoseries.GeoSeries'>
not counted
480
start
POINT (83.44175323902084 27.58080853478612)
481
POINT (83.43384305591235 27.59126572563281)
482
MULTIPOINT (83.42256136751314 27.59113847785852, 83.42489668023768 27.59244797169657)
483
end
MULTIPOINT (83.42705823674828 27.59324115665413, 83.43097265286571 27.5940181457209)
484
POINT (83.45582187967608 27.57507372177491)
485
start
POINT (83.44181823842104 27.59958052542095)
486
start
POINT (83.43629000058314 27.58906660983621)
487
POINT (83.43394270529166 27.58627037927465)
488
end
POINT (83.4524624306967 27.57971281519434)
489
end
POINT (83.44893423562655 27.59730571098981)
490
end
MULTIPOINT (83.42256136751314 27.59113847785852, 83.42489668023768 27.59244797169657)
491
POINT (83.43977384378792 27.58933656113345)
492
POINT (83.44640348583226 27.59578662385366)
493
start
POINT (83.44426803574129 27.58177434461038)
494
end
POINT (83.45303978285978 27.57960234607145)
495
POINT (8

POINT (83.42233495959078 27.57191745572607)
641
start
POINT (83.425918469262 27.56388758904103)
642
POINT (83.44564460371555 27.56825448465679)
643
POINT (83.43853386664466 27.58141989301373)
644
end
POINT (83.42669975798748 27.57077303604683)
645
<class 'geopandas.geoseries.GeoSeries'>
not counted
646
POINT (83.42485538696661 27.58555470148832)
647
POINT (83.42927889490511 27.56275239101006)
648
end
POINT (83.43970975450162 27.58614945643234)
649
end
POINT (83.45033619416012 27.57890155386366)
650
POINT (83.43979816145605 27.58925886960122)
651
start
POINT (83.42522274383464 27.56751184699419)
652
POINT (83.42866827232297 27.56269540816663)
653
POINT (83.42484580918682 27.56608408061589)
654
POINT (83.42504075252879 27.58467573199948)
655
POINT (83.42480350058116 27.58886350514672)
656
POINT (83.44476763521823 27.59921733151469)
657
end
POINT (83.43308950677539 27.58448811363559)
658
POINT (83.43124570502533 27.58942336010006)
659
POINT (83.45183228023558 27.57981031609324)
660
start


MULTIPOINT (83.46160463194826 27.59723321394171, 83.46162107065595 27.59762752978673)
804
POINT (83.45308427322067 27.60288050673006)
805
POINT (83.456998934853 27.60393661838026)
806
POINT (83.43782743748631 27.58054813608766)
807
end
POINT (83.45404105744888 27.6052695287658)
808
POINT (83.45593157538195 27.60180373495444)
809
start
POINT (83.45245941347122 27.60293250643008)
810
POINT (83.45292674887042 27.60290251853644)
811
POINT (83.43791902983941 27.57163115835135)
812
POINT (83.42615304212927 27.57165748870221)
813
POINT (83.42655215120887 27.57170509710377)
814
POINT (83.42639630604475 27.58852866416657)
815
end
POINT (83.46222015132285 27.58936818939479)
816
LINESTRING EMPTY
817
POINT (83.46244568961691 27.57812763148507)
818
POINT (83.46473460271977 27.58787808200793)
819
start
LINESTRING EMPTY
820
POINT (83.45112073303545 27.60051097749965)
821
<class 'geopandas.geoseries.GeoSeries'>
not counted
822
POINT (83.45112509744536 27.60052363545748)
823
POINT (83.42440234898925 27

In [16]:
assign_building_code()

0
1

2

3

4

5

6

7

8

9

10

11

12

13

14



C:\Users\Kshitiz\AppData\Local\Temp/ipykernel_21020/164222473.py:298: UserWarning: Geometry is in a geographic CRS. Results from 'interpolate' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always


15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

64

65

66

67

68

69

70

71

72

73

74

75

76

77

78

79

80

81

82

83

84

85

86

87

88

89

90

91

92

93

94

95

96

97

98

99

100

101

102

103

104

105

106

107

108

109

110

111

112

113

114

115

116

117

118

119

120

121

122

123

124

125

126

127

128

129

130

131

132

133

134

135

136

137

138

139

140

141

142

143

144

145

146

147

148

149

150

151

152

153

154

155

156

157

158

159

160

161

162

163

164

165

166

167

168

169

170

171

172

173

174

175

176

177

178

179

180

181

182

183

184

185

186

187

188

189

190

191

192

193

194

195

196

197

198

199

200

201

202

203

204

205

206

207

208

209

210

211

212

213

214

215

216

217

218

219

220

221

222

223

224

225

226

227

228

229

230

231



1573

1574

1575

1576

1577

1578

1579

1580

1581

1582

1583

1584

1585

1586

1587

1588

1589

1590

1591

1592

1593

1594

1595

1596

1597

1598

1599

1600

1601

1602

1603

1604

1605

1606

1607

1608

1609

1610

1611

1612

1613

1614

1615

1616

1617

1618

1619

1620

1621

1622

1623

1624

1625

1626

1627

1628

1629

1630

1631

1632

1633

1634

1635

1636

1637

1638

1639

1640

1641

1642

1643

1644

1645

1646

1647

1648

1649

1650

1651

1652

1653

1654

1655

1656

1657

1658

1659

1660

1661

1662

1663

1664

1665

1666

1667

1668

1669

1670

1671

1672

1673

1674

1675

1676

1677

1678

1679

1680

1681

1682

1683

1684

1685

1686

1687

1688

1689

1690

1691

1692

1693

1694

1695

1696

1697

1698

1699

1700

1701

1702

1703

1704

1705

1706

1707

1708

1709

1710

1711

1712

1713

1714

1715

1716

1717

1718

1719

1720

1721

1722

1723

1724

1725

1726

1727

1728

1729

1730

1731

1732

1733

1734

1735

1736

1737

1738

1739

2941

2942

2943

2944

2945

2946

2947

2948

2949

2950

2951

2952

2953

2954

2955

2956

2957

2958

2959

2960

2961

2962

2963

2964

2965

2966

2967

2968

2969

2970

2971

2972

2973

2974

2975

2976

2977

2978

2979

2980

2981

2982

2983

2984

2985

2986

2987

2988

2989

2990

2991

2992

2993

2994

2995

2996

2997

2998

2999

3000

3001

3002

3003

3004

3005

3006

3007

3008

3009

3010

3011

3012

3013

3014

3015

3016

3017

3018

3019

3020

3021

3022

3023

3024

3025

3026

3027

3028

3029

3030

3031

3032

3033

3034

3035

3036

3037

3038

3039

3040

3041

3042

3043

3044

3045

3046

3047

3048

3049

3050

3051

3052

3053

3054

3055

3056

3057

3058

3059

3060

3061

3062

3063

3064

3065

3066

3067

3068

3069

3070

3071

3072

3073

3074

3075

3076

3077

3078

3079

3080

3081

3082

3083

3084

3085

3086

3087

3088

3089

3090

3091

3092

3093

3094

3095

3096

3097

3098

3099

3100

3101

3102

3103

3104

3105

3106

3107

4312

4313

4314

4315

4316

4317

4318

4319

4320

4321

4322

4323

4324

4325

4326

4327

4328

4329

4330

4331

4332

4333

4334

4335

4336

4337

4338

4339

4340

4341

4342

4343

4344

4345

4346

4347

4348

4349

4350

4351

4352

4353

4354

4355

4356

4357

4358

4359

4360

4361

4362

4363

4364

4365

4366

4367

4368

4369

4370

4371

4372

4373

4374

4375

4376

4377

4378

4379

4380

4381

4382

4383

4384

4385

4386

4387

4388

4389

4390

4391

4392

4393

4394

4395

4396

4397

4398

4399

4400

4401

4402

4403

4404

4405

4406

4407

4408

4409

4410

4411

4412

4413

4414

4415

4416

4417

4418

4419

4420

4421

4422

4423

4424

4425

4426

4427

4428

4429

4430

4431

4432

4433

4434

4435

4436

4437

4438

4439

4440

4441

4442

4443

4444

4445

4446

4447

4448

4449

4450

4451

4452

4453

4454

4455

4456

4457

4458

4459

4460

4461

4462

4463

4464

4465

4466

4467

4468

4469

4470

4471

4472

4473

4474

4475

4476

4477

4478

5688

5689

5690

5691

5692

5693

5694

5695

5696

5697

5698

5699

5700

5701

5702

5703

5704

5705

5706

5707

5708

5709

5710

5711

5712

5713

5714

5715

5716

5717

5718

5719

5720

5721

5722

5723

5724

5725

5726

5727

5728

5729

5730

5731

5732

5733

5734

5735

5736

5737

5738

5739

5740

5741

5742

5743

5744

5745

5746

5747

5748

5749

5750

5751

5752

5753

5754

5755

5756

5757

5758

5759

5760

5761

5762

5763

5764

5765

5766

5767

5768

5769

5770

5771

5772

5773

5774

5775

5776

5777

5778

5779

5780

5781

5782

5783

5784

5785

5786

5787

5788

5789

5790

5791

5792

5793

5794

5795

5796

5797

5798

5799

5800

5801

5802

5803

5804

5805

5806

5807

5808

5809

5810

5811

5812

5813

5814

5815

5816

5817

5818

5819

5820

5821

5822

5823

5824

5825

5826

5827

5828

5829

5830

5831

5832

5833

5834

5835

5836

5837

5838

5839

5840

5841

5842

5843

5844

5845

5846

5847

5848

5849

5850

5851

5852

5853

5854

In [ ]:
buildings.to_file("output_building.geojson", driver="GeoJSON")

In [ ]:
print(buildings)

In [ ]:
road.to_file("output_road_1.geojson", driver="GeoJSON")